In [2981]:
import pandas as pd
import numpy as np

df = pd.read_csv('Messages.csv')
df_key = pd.read_csv('Keywords.csv')

In [2982]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [2983]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Initialize the PorterStemmer
stemmer = PorterStemmer()

corpus = []
for i in range(len(df)):
    # Remove non-letter characters and convert to lowercase
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower().split()
    
    # Apply stemming and remove stopwords
    review = [stemmer.stem(word) for word in review if word not in set(stopwords.words('english'))]
    
    # Join the words back into a single string
    review = ' '.join(review)
    
    # Add the processed review to the corpus
    corpus.append(review)


In [2984]:
from nltk import sent_tokenize

words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))


In [2985]:
words

[['desk'],
 ['flat', 'pack', 'desk', 'cupboard', 'pedest', 'drawer'],
 ['door', 'step', 'deliveri'],
 ['ground', 'floor'],
 ['number', 'afraid', 'wait', 'web'],
 ['day', 'bed'],
 ['ye', 'frame', 'singl', 'bed', 'mattress'],
 ['ground', 'floor', 'ground', 'floor', 'dismantl'],
 ['bed', 'frame', 'dismantl', 'mattress'],
 ['specif', 'time'],
 ['ye', 'pleas'],
 ['ok', 'wait', 'confirm', 'thank'],
 ['busi',
  'move',
  'larg',
  'printer',
  'requir',
  'four',
  'peopl',
  'lift',
  'shelv',
  'storag',
  'unit'],
 ['first', 'floor', 'lift', 'wide', 'staircas'],
 ['first', 'floor', 'lift'],
 ['men'],
 ['quot', 'two', 'peopl'],
 ['confirm', 'includ', 'pleas'],
 ['ok',
  'sorri',
  'think',
  'clear',
  'busi',
  'move',
  'main',
  'item',
  'would',
  'item',
  'quit',
  'larg',
  'heavi',
  'also',
  'forgot',
  'mention',
  'one',
  'printer',
  'round',
  'corner',
  'hous',
  'uh',
  'would',
  'also',
  'need',
  'add',
  'pick'],
 ['paper', 'drill', 'mainli', 'box', 'storag', 'unit',

In [2986]:
corpus_key=[]
for i in range(len(df_key)):
    # Remove non-letter characters and convert to lowercase
    review = re.sub('[^a-zA-Z]', ' ', df_key['keyword'][i])
    review = review.lower().split()
    
    # Apply stemming and remove stopwords
    review = [stemmer.stem(word) for word in review if word not in set(stopwords.words('english'))]
    
    # Join the words back into a single string
    review = ' '.join(review)
    corpus_key.append(review)

    # Add the processed review to the corpus
corpus_key

['basement',
 'bungalow',
 'deliveri',
 'downstair',
 'front door',
 'garag',
 'garden',
 'home',
 'level',
 'pavement',
 'pickup',
 'right',
 'stair',
 'storag',
 'street level']

In [2987]:
def find_sentences_with_similar_words(corpus, similar_words):
    """
    Search for sentences in the corpus that contain any of the similar words.

    Parameters:
    - corpus: List[str], the corpus of text (each element is a sentence or document).
    - similar_words: List[str], words similar to the search word.

    Returns:
    - Dict[str, List[str]], a dictionary where keys are similar words and values are lists of sentences containing those words.
    """
    # Initialize a dictionary to hold the results
    results = {word: [] for word in similar_words}
    
    # Iterate over the corpus to find sentences containing any of the similar words
    for sentence in corpus:
        for word in similar_words:
            if word in sentence.lower():  # Case-insensitive search
                results[word].append(sentence)
        
    counts = {word: len(sentences) for word, sentences in results.items()}
    
    data_for_df = [(word, "; ".join(sentences)) for word, sentences in results.items()]
 
    return results, counts , data_for_df


# Assuming 'model' is your trained Word2Vec model
#similar_words = [word for word, _ in model.wv.similar_by_word('delivery')]
similar_words=corpus_key
# Assuming 'corpus' is a list of sentences

# Find sentences containing similar words
sentences_with_similar_words , sentence_counts , data_for_df = find_sentences_with_similar_words(corpus, similar_words)

# Print the results
for word, sentences in sentences_with_similar_words.items():
    print(f"Sentences containing '{word}':")
    for sentence in sentences:
        print(f"- {sentence}")
    print()  # Blank line for readability

df_counts = pd.DataFrame(list(sentence_counts.items()), columns=['Word', 'Number of Sentences'])
df_sentences = pd.DataFrame(data_for_df, columns=['Word', 'Sentences'])



Sentences containing 'basement':
- lift pick first floor flat deliveri basement flat
- basement flat basement flat lift
- basement flat lift take stair one floor
- basement stair climb
- destin ground floor apart move duplex ground floor basement bedroom item basement stair
- end basement
- basement lift around steepish narrow stair

Sentences containing 'bungalow':
- bungalow
- go bungalow
- place gf dont know ladi carpet bungalow
- bungalow
- bungalow properti
- ground floor flat bungalow back
- ground floor flat bungalow
- ground floor one garag item ground floor bungalow collect sofa hous step go
- first floor flat go bungalow
- bungalow
- live bungalow stair daughter garag flat outsid hous
- need lift l said live bungalow everi thing flat
- bungalow stair
- nd floor hous straight intern stair main door destin bungalow
- detach bungalow
- bungalow
- move two bed flat bungalow
- hous collect bungalow
- ground level hous ground level bungalow
- ground floor bungalow
- ground floor bu

In [2993]:
df_counts_sorted = df_counts.sort_values(by='Number of Sentences', ascending=False)
df_counts_sorted

,Word,Number of Sentences
12,stair,201
2,deliveri,102
6,garden,51
13,storag,50
5,garag,41
3,downstair,31
4,front door,31
1,bungalow,30
8,level,25
11,right,24


In [2996]:
# Function to count occurrences of each keyword or phrase in the corpus
def count_keyword_occurrences(corpus, keywords):
    # Convert the entire corpus into a single string for easier searching
    corpus_text = ' '.join(corpus).lower()
    
    # Initialize a dictionary to hold the count of occurrences for each keyword
    keyword_occurrences = {}
    
    # Loop through each keyword to find and count its occurrences in the corpus text
    for keyword in keywords:
        # Normalize keyword to ensure consistent matching
        normalized_keyword = keyword.lower()
        # Count occurrences of the keyword in the corpus text
        count = corpus_text.count(normalized_keyword)
        # Store the count in the dictionary
        keyword_occurrences[keyword] = count
    
    return keyword_occurrences

# Assuming corpus and corpus_key are defined as before
# Normalize keywords since we're dealing with exact matches
keywords = [keyword.lower() for keyword in corpus_key]

# Count occurrences of each keyword or phrase in the corpus
keyword_occurrences = count_keyword_occurrences(corpus, keywords)

# keyword_occurrences now contains the count of each keyword or phrase present in the corpus


In [2994]:
# Assuming keyword_occurrences is the dictionary with keyword counts
keyword_occurrences_sorted = sorted(keyword_occurrences.items(), key=lambda x: x[1], reverse=True)

# Convert sorted list of tuples back to a dictionary if you prefer dictionary format
keyword_occurrences_sorted_dict = dict(keyword_occurrences_sorted)

# keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2995]:
keyword_occurrences_sorted_dict

{'stair': 224,
 'deliveri': 104,
 'garden': 58,
 'storag': 51,
 'garag': 42,
 'downstair': 33,
 'front door': 31,
 'bungalow': 30,
 'level': 30,
 'right': 24,
 'home': 23,
 'pickup': 17,
 'basement': 9,
 'street level': 8,
 'pavement': 3}